## Tratamento dos dados

#### Importação das bibliotecas

In [2]:
import pandas as pd
import os
from graph_tool import Graph, stats
from netpixi.data.gt import *
import netpixi

#### Ligas disponíveis

O dataset de transferências possui os dados das seguintes ligas:

#### Problemas:

- Sub21 e Sub18
- Incoerência entre as colunas club_name e club_involved_name

In [3]:
DATA_DIR = './data/transfers_master/data'
os.listdir(DATA_DIR + '/' + '2014')

FileNotFoundError: [Errno 2] No such file or directory: './data/transfers_master/data/2014'

#### Loading Data

Função para carregar o dataset. Recebe o diretório que o data está salvo e o ano.

In [ ]:
def loading_data(DATA_DIR, year):
    
    # Leitura do primeiro dataset para carregar as columns
    dir = "./data/transfers_master/data/"+ year + "/dutch_eredivisie.csv"
    data = pd.read_csv(dir)

    # Loop desconsidera o primeiro elemento (carregado no código acima)
    for league in os.listdir(DATA_DIR + '/' + year)[1:]:
            string = (DATA_DIR + '/' + year + '/' + league)
            data_temp = pd.read_csv(string)
            
            # Concatenação dos dados
            data = pd.concat([data, data_temp])
     
    return(data)

def concat_year(start, end, DATA_DIR, data):
    
    for year in range(start, end+1):
        data = pd.concat([data, loading_data(DATA_DIR, str(year))])
        
    return data

In [ ]:
DATA_DIR = './data/transfers_master/data'
YEAR = '1992'

data = loading_data(DATA_DIR, YEAR)
data

In [ ]:
data = concat_year(1993, 2015, DATA_DIR, data)

In [ ]:
data_in = data.loc[data["transfer_movement"] == "in"]
data_out = data.loc[data["transfer_movement"] == "out"]
data_in = data_in.reset_index()
data_out = data_out.reset_index()

In [ ]:
import matplotlib.pyplot as plt

pre_gal = pl.sort_index()
pos_gal = pl.sort_index()
pre_gal = pre_gal[:11]
pos_gal = pos_gal[11:]

plt.style.use('ggplot')
pl = data_in["year"].value_counts()
plt.title("Número Total de Transferências")
plt.xlabel("Ano")
plt.ylabel("Transferências")
plt.plot(pl.sort_index(), 'o', label = "Transferências")
plt.plot(2003 ,pl[2003], 'bo', label = "Era")

In [ ]:
plt.figure(figsize = (8,4))
plt.plot(pre_gal, 'o', label = "Pré-Galácticos", color = "#AA33AA")
plt.plot(pos_gal, 'o', label = "Pós-Galácticos", color = "#3333AA")
plt.title("Número Total de Transferências")
plt.xlabel("Ano")
plt.ylabel("Transferências")
plt.legend(loc="upper left")
plt.show()

In [ ]:
in_names = list(data_in["club_name"].value_counts().index)
out_names = list(data_in["club_involved_name"].value_counts().index)
name_dic = {}
problems = ["Olympique Marseille", "Olympique Lyon", "Eintracht Braunschweig", "Eintracht Frankfurt"]
for in_str in in_names:
    for out_str in out_names:

        if out_str[:10] in in_str and out_str != in_str:
            if (out_str not in problems) or (in_str not in problems):
                print(in_str ,", ", out_str)
                name_dic[out_str] = in_str

In [ ]:
for k,v in name_dic.items():
    data_in["club_involved_name"].replace({k: v}, inplace=True)
    
data_in

In [ ]:
data_in["year"].value_counts()

In [ ]:
data_pre_gal = data_in[data_in["year"] <= 2003]
data_pos_gal =  data_in[data_in["year"] > 2003]

In [ ]:
g = Graph(directed=True) # dirigido
g.vp['name'] = g.new_vp('object')

In [ ]:
times = []
for index, rows in data_pre_gal.iterrows():
    time1 = rows[1]
    time2 = rows[5]
    if time1 not in times:
        times.append(time1)
        g.add_vertex()
        v_time = g.vertex(times.index(time1))
        g.vp['name'][v_time] = time1
        
    if time2 not in times:
        times.append(time2)
        g.add_vertex()
        v_time = g.vertex(times.index(time2))
        g.vp['name'][v_time] = time2
    
    index1 = times.index(time1)
    index2 = times.index(time2)
    
    if g.edge(index1, index2) == None:
        g.add_edge(index1, index2)

In [ ]:
m = draw_gt.sfdp_layout(g)
move_gt(g, m)
save_gt(g, 'pre_gal.net.gz')

In [ ]:
r = netpixi.render('pre_gal.net.gz')

In [ ]:
# mudar a cor de fundo do grafo
r.graph(color=0xffffff)

# mudar a cor e o tamanho dos vértices
r.vertex_default(color=0x7E3C6B, size=10)

# mudar a cor, a espessura e a curvatura das arestas
r.edge_default(color=0x000000, width=1, curve1=0, curve2=0)
r.vertex_set_key('name')

In [ ]:
g2 = Graph(directed=True) # dirigido
g2.vp['name'] = g2.new_vp('object')

In [ ]:
times = []
for index, rows in data_pos_gal.iterrows():
    time1 = rows[1]
    time2 = rows[5]
    if time1 not in times:
        times.append(time1)
        g2.add_vertex()
        v_time = g2.vertex(times.index(time1))
        g2.vp['name'][v_time] = time1
        
    if time2 not in times:
        times.append(time2)
        g2.add_vertex()
        v_time = g2.vertex(times.index(time2))
        g2.vp['name'][v_time] = time2
    
    index1 = times.index(time1)
    index2 = times.index(time2)
    
    if g.edge(index1, index2) == None:
        g.add_edge(index1, index2)

In [ ]:
stats.remove_self_loops(g2)
m = draw_gt.sfdp_layout(g2)
move_gt(g2, m)
save_gt(g2, 'pos_gal.net.gz')

In [ ]:
r = netpixi.render('pos_gal.net.gz')

In [ ]:
# mudar a cor de fundo do grafo
r.graph(color=0xffffff)

# mudar a cor e o tamanho dos vértices
r.vertex_default(color=0x7E3C6B, size=10)

# mudar a cor, a espessura e a curvatura das arestas
r.edge_default(color=0x000000, width=1, curve1=0, curve2=0)
r.vertex_set_key('name')